# Lab 03 - Working with and Visualizing Geospatial Data

In this notebook, we will learn about `geopandas` - a Python library specialized in reading and processing geospatial vector data files such as **Shapefiles** and **GeoJSON** files.

First, ensure that you have `geopandas` installed in your environment. Note that `geopandas` can be quite fussy with environments and operating systems.

The suggested way for Windows-based installations is to ensure that all your libraries are using the same channel (i.e. `conda-forge`)

```bash
conda install -c conda-forge geopandas
```

For those using `pip`, you may also install simply using 
```bash
pip install geopandas
```

In [ ]:
# # Uncomment this cell if running on Google Colab ONLY!
# !pip install geopandas

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

from pathlib import Path

%pylab inline

In [ ]:
## Only do this if you're working on Google Colab

# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

In [ ]:
# when using Google Colab
# dataset_folder = Path('/gdrive/MyDrive/datasets')

# when using local folder
dataset_folder = Path('datasets/')

## Shapefiles

As defined by ArcGIS:
> A shapefile is an Esri vector data storage format for storing the location, shape, and attributes of geographic features. It is stored as a set of related files and contains one feature class.
>
> The primary way to make shapefile data available for others to view through a web browser is to add it to a `.zip` file, upload it, and publish a hosted feature layer. The `.zip` file must contain at least the `.shp`, `.shx`, `.dbf`, and `.prj` files components of the shapefile.

For more details, see the [main documentation](https://doc.arcgis.com/en/arcgis-online/reference/shapefiles.htm).

### GADM Dataset

Go to [GADM](https://gadm.org/download_country.html) and download the [Philippines shapefiles](https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_PHL_shp.zip).

Unzip the file into a folder and make sure all files are within the folder.

In [ ]:
ph_gdf = gpd.read_file(dataset_folder / 'gadm41_PHL_shp/gadm41_PHL_1.shp')
ph_gdf.head(20)

### Education Facilities Point Data

The provided files were downloaded from the Humanitarian Data Exchange website as extracted by the Humanitarian OpenStreetMap Team (HOTOSM):
* [PHL South Education Facilities](https://data.humdata.org/dataset/hotosm_phl_south_education_facilities)
* [PHL South Education Facilities](https://data.humdata.org/dataset/hotosm_phl_north_education_facilities)

To simplify the dataset for use in the Plotly Dash app, we'll concatenate these two dataframes.

In [ ]:
educ_s = gpd.read_file(dataset_folder / 'hotosm_phl_south_education_facilities_points_shp/hotosm_phl_south_education_facilities_points.shp')
educ_s.head()

In [ ]:
educ_s.shape

In [ ]:
educ_n = gpd.read_file(dataset_folder / 'hotosm_phl_north_education_facilities_points_shp/hotosm_phl_north_education_facilities_points.shp')
educ_n.head()

In [ ]:
educ_n.shape

In [ ]:
educ_sites = pd.concat([educ_n, educ_s])
educ_sites.head()

In [ ]:
educ_sites.shape

## Visualizing geospatial data using `geopandas`

`geopandas` provides a simple and straightforward way to visualize the geographic boundaries and any underlying column information. 

However, note that there are some data transformations that would be needed for creating choropleth maps.

In [ ]:
ax = ph_gdf.plot(figsize=(20, 14), color='white', edgecolor='dimgray')

### Figure 1. Base map of the administrative boundaries of the Philippines (provincial level)

This visualization only shows the geometries of the boundaries of each province in the Shapefile. The dataset does not have any quantitative or categorical attribute that can be visualized. This shapefile is usually used as a base for creating choropleth maps.

In [ ]:
ax = ph_gdf.plot(figsize=(20, 14), color='white', edgecolor='dimgray')
educ_sites.plot(ax=ax, column='amenity', legend=True, alpha=0.3)

### Figure 2. Scatter Map of the Education Facilities in the Philippines

This is a two-layer visualization of the education facilities dataset on top of the administrative boundaries dataset.

In `geopandas`, visualizing the administrative boundaries provides the context for the scatter points. Without it, the graph would look like the next block.

In [ ]:
educ_sites.plot(figsize=(20,14), column='amenity', legend=True, alpha=0.3)

### Save the merged dataset for a single loading in the Plotly Dash application

`geopandas` provides a way to save any changes you've made to the `GeoDataFrame`. You may save it as a Shapefile or a GeoJSON.

In [ ]:
# shapefile
educ_sites.to_file(dataset_folder / 'hotosm_phl_education_facilities.shp')

# geojson
educ_sites.to_file(dataset_folder / 'hotosm_phl_education_facilities.geojson', driver='GeoJSON')

## How to make a choropleth map?

Given the dataset that we have, it is not ready for a choropleth map. But we can **derive data** from the current point data we have and the polygon administrative boundaries.

A common structure for the dataset to easily make a choropleth map would be to have:
* each location should be **ONE** row.
* each quantitative attribute about that location is a **COLUMN**.

This way, it's easy to access the location as a *key* from the row and also visualize and entire column variable for a map. It also prepares your data for easy interactions later by selecting only the column needed for the visual.

### How to transform the data?
**First**, identify the administrative boundary level for aggregation. Make sure that you have `Polygon` type geometry for the data.

In [ ]:
ph_gdf.plot(figsize=(20, 14), color='white', edgecolor='dimgray')

In [ ]:
ph_gdf.head(10)

In our case, since we already have the provincial level administrative boundaries, we'll use this for simplicity.

**Next**, determine how you can aggregate your observation data (usually `Point`) or with longitude and latitude values into the regions you've identified.

In [ ]:
educ_sites.head(10)

Looking at our education facilities dataset, we don't have a column that matches the `NAME_1` attribute of `ph_gdf`. We don't have a text field that can be used for a `pandas` merge.

Luckily, since we're working with two `GeoDataFrame`s with geometries, we can perform spatial join.

We can check whether the `Polygon` **contains** the `Points` or if they simply **intersect**.

By performing a spatial join, we're assigning a province value to the educational facility in the dataset.

This is achieved using the `gdp.sjoin()` method. [Documentation](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html)

In [ ]:
ph_educ_site_intersect = gpd.sjoin(ph_gdf, # left geodataframe (the polygons will be kept)
                               educ_sites, # right geodataframe (the ponit geometry will be dropped, but all attributes will be kept)
                               how='left', # default is 'inner' but we want to keep all the polygons! we don't want a map with a hole!
                               predicate='intersects' # default
                              )
ph_educ_site_intersect.head()

In [ ]:
ph_educ_site_intersect.shape

In [ ]:
educ_sites.shape

### WAIT! Why do we have more eductional facilities?!

Since we used the predicate or spatial operation **intersects**, if the point falls on the border or a corner of two or more polygons, then that point will be assigned to all the polygons it intersects with.

Let's try contains to see if we're able to keep the same number of facilities.

In [ ]:
ph_educ_site_contains = gpd.sjoin(ph_gdf, # left geodataframe (the polygons will be kept)
                               educ_sites, # right geodataframe (the ponit geometry will be dropped, but all attributes will be kept)
                               how='left', # default is 'inner' but we want to keep all the polygons! we don't want a map with a hole!
                               predicate='contains' # points should be within the polygon
                              )
ph_educ_site_contains.head()

In [ ]:
ph_educ_site_contains.shape

### Duplicated data problem

It seems like whether we use **contains** or **intersects**, the output is the same. How can we fix this?

We can opt to drop_duplicates based on the `osm_id` subset.

In [ ]:
ph_educ_site_contains[ph_educ_site_contains.duplicated(subset='osm_id', keep=False)]

Looking at the patterns, we can see that the school names are repeating, but the `NAME_1` is also identical. From this, we can potentially deduce that there may already be duplicates from the original dataset.

In [ ]:
educ_sites.duplicated(subset='osm_id').sum()

Let's drop the duplicates and re-run the `sjoin` (for contains)

In [ ]:
educ_sites = educ_sites.drop_duplicates()
educ_sites.shape

In [ ]:
ph_educ_site_contains = gpd.sjoin(ph_gdf, # left geodataframe (the polygons will be kept)
                               educ_sites, # right geodataframe (the ponit geometry will be dropped, but all attributes will be kept)
                               how='left', # default is 'inner' but we want to keep all the polygons! we don't want a map with a hole!
                               predicate='contains' # points should be within the polygon
                              )
ph_educ_site_contains.shape

The dataset seems to be now missing **9 rows**. This may warrant further investigation for data cleaning and processing. But for the purpose of this lesson, we'll focus on creating the dataset for the choropleth map.

In [ ]:
ph_educ_site_contains.head(10)

In [ ]:
ph_educ_site_contains.geometry.head()

As mentioned in the comments, the `geometry` of the left `GeoDataFrame` will be the one left. The Point information is now dropped.

This is because a `GeoDataFrame` can only have **ONE** geometry information.

But looking at the dataset, we can see that the location information is now repeating(!) but what we want is **ONE** row per location (`NAME_1`).

We can now work with `groupby` to get the data that we need.

But first, let's check if we can create more columns than just `value_counts` per location.

In [ ]:
ph_educ_site_contains.columns

### Extracting categorical variables for counts

When working with geospatial data, usually locations have categories that can be used for filtering! So in our case, we can try to extract the information about the following:
* amenity
* building
* operatory

In [ ]:
ph_educ_site_contains.amenity.unique()

In [ ]:
ph_educ_site_contains.building.unique()

In [ ]:
ph_educ_site_contains.operatorty.unique()

Looking at all of these, it seems like some further data cleaning may need to be done for the `amenity` column.

The `building` column is a bit tricky to use since there are values like *house* and *yes*.

For the `operatory`, most of the values are quite appropriate, so we can also create categories from this.

In [ ]:
ph_educ_site_contains[ph_educ_site_contains.amenity == 'bus_station']

In [ ]:
clean_ph_educ_site = ph_educ_site_contains[ph_educ_site_contains.amenity != 'bus_station']
clean_ph_educ_site.shape

In [ ]:
amenity_group = clean_ph_educ_site.groupby(['NAME_1','amenity'])['osm_id'].count().reset_index()
amenity_group.head()

In [ ]:
operatorty_group = clean_ph_educ_site.groupby(['NAME_1','operatorty'])['osm_id'].count().reset_index()
operatorty_group.head()

Now that we have the counts per category, we can now create pivot tables to be used for merging.

In [ ]:
amenity_pivot = pd.pivot_table(amenity_group, index='NAME_1', columns='amenity', values='osm_id', fill_value=0)
amenity_pivot.head()

In [ ]:
operatorty_pivot = pd.pivot_table(operatorty_group, index='NAME_1', columns='operatorty', values='osm_id', fill_value=0)
operatorty_pivot.head()

Since there's two `university` values for both, let's keep them as separate `DataFrames` instead of merging them.

What we need to do next is to **merge this with the `ph_gdf` `GeoDataFrame`** so that we have a geospatial dataset to work with instead of just a regular `DataFrame`.

In [ ]:
# IMPORTANT: The GeoDataFrame should be on the OUTSIDE (left) of the merge so you keep the GeoDataFrame type
amenity_gdf = ph_gdf.merge(amenity_pivot, left_on='NAME_1', right_index=True, how='left')
amenity_gdf.fillna(0, inplace=True)
amenity_gdf.head()

In [ ]:
# IMPORTANT: The GeoDataFrame should be on the OUTSIDE (left) of the merge so you keep the GeoDataFrame type
operatorty_gdf = ph_gdf.merge(operatorty_pivot, left_on='NAME_1', right_index=True, how='left')
operatorty_gdf.fillna(0, inplace=True)
operatorty_gdf.head()

In [ ]:
print(ph_gdf.shape)
print(amenity_gdf.shape)
print(operatorty_gdf.shape)

In [ ]:
amenity_gdf.columns

In [ ]:
amenity_gdf = amenity_gdf[['GID_1', 'GID_0', 'COUNTRY', 'NAME_1', 'geometry', 'college',
       'kindergarten', 'school', 'university']]
amenity_gdf.columns = ['gid_1', 'gid_0', 'country', 'province', 'geometry', 'college',
       'kindergarten', 'school', 'university']

In [ ]:
operatorty_gdf.columns

In [ ]:
operatorty_gdf = operatorty_gdf[['GID_1', 'GID_0', 'COUNTRY', 'NAME_1', 'geometry',
       'consortium', 'corporation', 'government', 'private', 'public',
       'religious', 'university']]
operatorty_gdf.columns = ['gid_1', 'gid_0', 'country', 'province', 'geometry',
       'consortium', 'corporation', 'government', 'private', 'public',
       'religious', 'university']

In [ ]:
amenity_gdf.plot(figsize=(20, 14), column='kindergarten', legend=True)
plt.title('Kindergarten');

In [ ]:
operatorty_gdf.plot(figsize=(20, 14), column='religious', legend=True)
plt.title('Operatorty: Religious');

In [ ]:
operatorty_gdf.plot(figsize=(20, 14), column='public', legend=True)
plt.title('Operatorty: Public');

In [ ]:
operatorty_gdf.plot(figsize=(20, 14), column='private', legend=True)
plt.title('Operatorty: Private');

## Conclusion

When making choropleth maps, it's not usually the case that you would already have the data in the right structure.

It might be possible when working with time series data. In that case, you would need to have:
* one location for each row
* one year/month/day for each column (also achieved by creating a pivot table > merge)

## Save the transformed data

One of the reasons why we chose to drop the unnecessary columns is to make the dataset file as small as possible so that when it's used for the web visualization, it would not cause a big slowdown to the application.

In [ ]:
amenity_gdf.to_file(dataset_folder / 'ph_educ_by_amenity.geojson', driver='GeoJSON')

operatorty_gdf.to_file(dataset_folder / 'ph_educ_by_operatorty.geojson', driver='GeoJSON')